#从航空机票查询数据提取特征如下：
1.查询票价日期对应的年、月、日以及是一周的第几天；
2.航班起飞日期对应的年、月、日以及是一周的第几天；
3.查询票价日期与航班起飞日期相差的天数；
4.查询票价日期与航班起飞日期区间内最低票价（label）；
5.查询票价日期与航班起飞日期区间内平均最低票价；
6.上月平均最低票价
7.当前月所在季度平均最低票价

In [83]:
import pandas as pd
import numpy as np

In [84]:
fd = pd.read_csv('valid_data.csv')   #读取清洗好的数据，没有缺失值，remain_day<=60的数据
print(fd.columns)
print(fd.head(10))


Index(['ticket_date', 'flight_date', 'remain_day', 'valid_price'], dtype='object')
  ticket_date flight_date  remain_day  valid_price
0   2017/7/21    2017/8/9          19          920
1   2017/7/21   2017/9/13          54          920
2   2017/7/21   2017/9/14          55          920
3   2017/7/21    2017/9/5          46          530
4   2017/7/21    2017/9/9          50          530
5   2017/7/21   2017/9/11          52          530
6   2017/7/21   2017/9/12          53          530
7   2017/7/21   2017/9/16          57          530
8   2017/7/21   2017/9/18          59          530
9   2017/7/21   2017/7/24           3         1040


In [85]:
#获取查票查票日期和航班日期对应的年月日以及是周几
fd['ticket_year'] = pd.to_datetime(fd['ticket_date']).dt.year               # f1:年
fd['ticket_month'] = pd.to_datetime(fd['ticket_date']).dt.month             # f1:月
fd['ticket_day'] = pd.to_datetime(fd['ticket_date']).dt.day                 # f1:日
fd['ticket_weekday']=pd.to_datetime(fd['ticket_date']).dt.weekday + 1       # f1:星期,+1的原因是星期一默认为0
fd['flight_year'] = pd.to_datetime(fd['flight_date']).dt.year               # f2:年
fd['flight_month'] = pd.to_datetime(fd['flight_date']).dt.month             # f2:月
fd['flight_day'] = pd.to_datetime(fd['flight_date']).dt.day                 # f2:日
fd['flight_weekday']=pd.to_datetime(fd['flight_date']).dt.weekday + 1       # f2:星期  
print(len(fd))
print(fd.head(10))

23612
  ticket_date flight_date  remain_day  valid_price  ticket_year  ticket_month  \
0   2017/7/21    2017/8/9          19          920         2017             7   
1   2017/7/21   2017/9/13          54          920         2017             7   
2   2017/7/21   2017/9/14          55          920         2017             7   
3   2017/7/21    2017/9/5          46          530         2017             7   
4   2017/7/21    2017/9/9          50          530         2017             7   
5   2017/7/21   2017/9/11          52          530         2017             7   
6   2017/7/21   2017/9/12          53          530         2017             7   
7   2017/7/21   2017/9/16          57          530         2017             7   
8   2017/7/21   2017/9/18          59          530         2017             7   
9   2017/7/21   2017/7/24           3         1040         2017             7   

   ticket_day  ticket_weekday  flight_year  flight_month  flight_day  \
0          21               5 

In [87]:
#将remain_day划分六个区间：0-10 == 1，11-20 == 2, 21-30 == 3, 31-40 == 4, 41-50 == 5, 51-60 == 6
import math
def remain_division(x):
    return math.ceil(x/10)

fd['remain_day_interval'] = fd['remain_day'].apply(remain_division)        #f3:remain_day，f4:remain_day所在区间

In [89]:
#提取航班数据有多少个航班期
flight_day = set(fd.flight_date)

#计算各航班期不同remain_day的最低票价
frames = []
for i in flight_day:
    data_list = []
    for j in range(len(fd)):
        if fd.loc[j,'flight_date'] == i:
            data_list.append(j)
    temp_date_data = fd.loc[data_list]
    temp_date_data.reset_index(drop = True, inplace = True)
    order_temp_date_data = temp_date_data.sort_values(['remain_day'],ascending = True)
    order_temp_date_data.reset_index(drop = True, inplace = True)
    order_temp_date_data['lowest_price'] = -1
    order_temp_date_data['ave_price'] = -1
    for k in range(0,len(order_temp_date_data)):   
        #提取各remain_day到航班日的最低价格
        order_temp_date_data.loc[k,'lowest_price'] = min(order_temp_date_data.loc[:k,'valid_price'])  #label
        #提取各remain_day到航班日的低价均值
        order_temp_date_data.loc[k,'ave_price'] = order_temp_date_data.loc[:k,'valid_price'].mean()   #f5:最小remain_day到当前remain_day的均价
    frames.append(order_temp_date_data)
result_pool = pd.concat(frames)
result_pool.reset_index(drop = True, inplace = True)
#result_pool.columns = ['ticket_date','flight_date','remain_day','valid_price','ticket_year','ticket_month','ticket_day','ticket_weekday',\
#                      'flight_year','flight_month','flight_day','flight_weekday','lowest_price','ave_price']
print(result_pool.shape)
print(result_pool.head(10))

(23612, 15)
  ticket_date flight_date  remain_day  valid_price  ticket_year  ticket_month  \
0   2017/8/11   2017/8/12           1         1020         2017             8   
1   2017/8/10   2017/8/12           2         1060         2017             8   
2    2017/8/9   2017/8/12           3         1060         2017             8   
3    2017/8/8   2017/8/12           4         1060         2017             8   
4    2017/8/7   2017/8/12           5         1060         2017             8   
5    2017/8/6   2017/8/12           6          680         2017             8   
6    2017/8/5   2017/8/12           7          550         2017             8   
7    2017/8/4   2017/8/12           8          550         2017             8   
8    2017/8/3   2017/8/12           9          550         2017             8   
9    2017/8/1   2017/8/12          11          550         2017             8   

   ticket_day  ticket_weekday  flight_year  flight_month  flight_day  \
0          11           

In [96]:
#获取每月各remain_day区间的均价
ave_price_month = result_pool.groupby(['flight_month','remain_day_interval']).agg('mean')
ave_price_month = pd.DataFrame(ave_price_month)
ave_price_month.reset_index(inplace = True)
ave_price_month = ave_price_month[['flight_month','remain_day_interval','valid_price','lowest_price','ave_price']]
ave_price_month.head(10)


result_pool['ave_valid_price'] = -1                                #f6:同月同remain_day区间的均价
result_pool['ave_lowest_price'] = -1                               #f7:同月同remain_day区间的最低均价
result_pool['ave_lowest_price_lastmonth'] = -1                     #f8:上月最低平均票价
for i in range(len(result_pool)):
    mon_id = result_pool.loc[i,'flight_month']
    interval_id = result_pool.loc[i,'remain_day_interval']
    index_id = ave_price_month[(ave_price_month.flight_month == mon_id)&(ave_price_month.remain_day_interval == interval_id)].index.tolist()
    index_id = index_id[0]
    #a[3] = ave_price_month.loc[index_id,'valid_price']
    result_pool.loc[i,'ave_valid_price'] = ave_price_month.loc[index_id,'valid_price']    #获取订票数据日期所在月的平均票价
    result_pool.loc[i,'ave_lowest_price'] = ave_price_month.loc[index_id,'lowest_price']  #获取订单数据日期所在月的最低票价
    if mon_id == 1:
        index_id = ave_price_month[(ave_price_month.flight_month == 12)&(ave_price_month.remain_day_interval == interval_id)].index.tolist()
        index_id = index_id[0]
        result_pool.loc[i,'ave_lowest_price_lastmonth'] = ave_price_month.loc[index_id,'lowest_price']    #获取订票数据日期所在月的上月平均最低票价
    else:
        index_id = ave_price_month[(ave_price_month.flight_month == (mon_id - 1))&(ave_price_month.remain_day_interval == interval_id)].index.tolist()
        index_id = index_id[0]
        result_pool.loc[i,'ave_lowest_price_lastmonth'] = ave_price_month.loc[index_id,'lowest_price']    #获取订票数据日期所在月的上月平均最低票价
    

In [5]:

#result_pool['remain_day_interval'] = result_pool['remain_day_interval'].astype(str)
#result_pool = pd.get_dummies(result_pool,columns = ['remain_day_interval'])
#result_pool.head(10)

,ticket_date,flight_date,remain_day,valid_price,ticket_year,ticket_month,ticket_day,ticket_weekday,flight_year,flight_month,flight_day,flight_weekday,lowest_price,ave_price,remain_day_interval_1,remain_day_interval_2,remain_day_interval_3,remain_day_interval_4,remain_day_interval_5,remain_day_interval_6
0,2017/12/10,2017/12/11,1,550,2017,12,10,7,2017,12,11,1,550,550.000000,1,0,0,0,0,0
1,2017/12/9,2017/12/11,2,560,2017,12,9,6,2017,12,11,1,550,555.000000,1,0,0,0,0,0
2,2017/12/9,2017/12/11,2,550,2017,12,9,6,2017,12,11,1,550,553.333333,1,0,0,0,0,0
3,2017/12/8,2017/12/11,3,550,2017,12,8,5,2017,12,11,1,550,552.500000,1,0,0,0,0,0
4,2017/12/7,2017/12/11,4,1060,2017,12,7,4,2017,12,11,1,550,654.000000,1,0,0,0,0,0
5,2017/12/7,2017/12/11,4,550,2017,12,7,4,2017,12,11,1,550,636.666667,1,0,0,0,0,0
6,2017/12/6,2017/12/11,5,550,2017,12,6,3,2017,12,11,1,550,624.285714,1,0,0,0,0,0
7,2017/12/5,2017/12/11,6,550,2017,12,5,2,2017,12,11,1,550,615.000000,1,0,0,0,0,0
8,2017/12/5,2017/12/11,6,530,2017,12,5,2,2017,12,11,1,530,605.555556,1,0,0,0,0,0
9,2017/12/4,2017/12/11,7,530,2017,12,4,1,2017,12,11,1,530,598.000000,1,0,0,0,0,0


In [97]:
result_pool.head(20)

,ticket_date,flight_date,remain_day,valid_price,ticket_year,ticket_month,ticket_day,ticket_weekday,flight_year,flight_month,flight_day,flight_weekday,remain_day_interval,lowest_price,ave_price,ave_valid_price,ave_lowest_price,ave_lowest_price_lastmonth
0,2017/8/11,2017/8/12,1,1020,2017,8,11,5,2017,8,12,6,1,1020,1020.000000,855.731922,789.506173,797.694444
1,2017/8/10,2017/8/12,2,1060,2017,8,10,4,2017,8,12,6,1,1020,1040.000000,855.731922,789.506173,797.694444
2,2017/8/9,2017/8/12,3,1060,2017,8,9,3,2017,8,12,6,1,1020,1046.666667,855.731922,789.506173,797.694444
3,2017/8/8,2017/8/12,4,1060,2017,8,8,2,2017,8,12,6,1,1020,1050.000000,855.731922,789.506173,797.694444
4,2017/8/7,2017/8/12,5,1060,2017,8,7,1,2017,8,12,6,1,1020,1052.000000,855.731922,789.506173,797.694444
5,2017/8/6,2017/8/12,6,680,2017,8,6,7,2017,8,12,6,1,680,990.000000,855.731922,789.506173,797.694444
6,2017/8/5,2017/8/12,7,550,2017,8,5,6,2017,8,12,6,1,550,927.142857,855.731922,789.506173,797.694444
7,2017/8/4,2017/8/12,8,550,2017,8,4,5,2017,8,12,6,1,550,880.000000,855.731922,789.506173,797.694444
8,2017/8/3,2017/8/12,9,550,2017,8,3,4,2017,8,12,6,1,550,843.333333,855.731922,789.506173,797.694444
9,2017/8/1,2017/8/12,11,550,2017,8,1,2,2017,8,12,6,2,550,814.000000,888.108696,746.978261,783.003413


In [98]:
#col = [ticket_date,flight_date,remain_day,valid_price,ticket_year,ticket_month,ticket_day,ticket_weekday,flight_year,flight_month,flight_day,\
#      flight_weekday,remain_day_interval,lowest_price,ave_price,ave_valid_price,ave_lowest_price,ave_lowest_price_lastmonth]
result_pool.to_csv('sample_data.csv',index = False)